# Yelp Fusion API - Training Data Pull 1/15/19

## Britt Allen, Bernard Kurka, Thomas Ludlow - NY-DSI-6

Figure out how to pull `price` and supporting data directly from Yelp using *Fusion API*.  

### Resources

GitHub: 
 - https://github.com/Yelp/yelp-python
 - https://github.com/gfairchild/yelpapi *(Best library)*
  - https://github.com/gfairchild/yelpapi/blob/master/examples/examples.py

Endpoint Documentation: https://www.yelp.com/developers/documentation/v3/business

Using regular search, a location-based query is formatted like this:
`https://www.yelp.com/search?find_loc=10128`

```
My App
Client ID
ea2TodAq4YX-4W3lzSJrcA

API Key
21Pt2l8__qgIdL0ZpgYC_yWblJ_O8_vJ3_-tIybHDyuQl9oVBXAzAXQWqMmIrz7idLyc7owv4-lfSON0QjKJN4pvQei4rUQAGSZcGcVTQc4HtBseUcztUPkVrAItXHYx
```

### Libraries

In [1]:
import numpy as np
import pandas as pd
import json
import time
from yelpapi import YelpAPI


## Search query to dataframe

In [2]:
def query_to_df(loc_in, cat_in=['restaurants','shopping','localservices'], 
                sort_in='distance', limit_in=50, 
                cols=['categories','alias','city','state','zip_code','price','review_count','latitude','longitude']):
    """Available arguments:
    loc_in (str): location (zip, city, neighborhood, etc.)
    cat_in (list): categories - default is ['restaurants','shopping','localservices']
    sort_in (str): sort criterion of 'distance','best_match','review_count' - default is 'distance'
    limit_in (int): number of results to pull per category, max is 50 - default is 50
    cols (list): columns for dataframe, matching API results key names - default is
    ['categories','alias','city','state','zip_code','price','review_count','latitude','longitude']
    """
    
    # Set Yelp Fusion API Key and establish API connection
    api_key = '21Pt2l8__qgIdL0ZpgYC_yWblJ_O8_vJ3_-tIybHDyuQl9oVBXAzAXQWqMmIrz7idLyc7owv4-lfSON0QjKJN4pvQei4rUQAGSZcGcVTQc4HtBseUcztUPkVrAItXHYx'
    api_obj = YelpAPI(api_key, timeout_s=3.0)
    
    # Instantiate empty DataFrame with desired output columns
    output_df = pd.DataFrame(columns=['search_term']+cols)
    
    # Create iterable list of limit amounts <= 50 so that full limit argument is covered
    # ex. 70 -> [50,20]
    limit_list = []
    if limit_in > 50:
        req = limit_in  # req starts at limit argument and counts down by 50 until < 50
        while req > 50:
            limit_list.append(50)
            req -= 50
        limit_list.append(req)
    else:
        limit_list.append(limit_in) # if req < 50 append remaining amount to list
    
    # Loop through category argument list items
    for cat in cat_in:
        cat_df = pd.DataFrame(columns=['search_term']+cols) # Create empty DataFrame with addl col for category
        for j, limit in enumerate(limit_list): # Perform API pulls with all limits in limit_list
            
            # API call saved to json dict
            if cat=='none':
                response = api_obj.search_query(location=loc_in, sort_by=sort_in, limit=limit, offset=(j*50))
            else:
                response = api_obj.search_query(location=loc_in, categories=[cat], sort_by=sort_in, limit=limit, offset=(j*50))
            response_df = pd.DataFrame(response['businesses']) # Save business data to DataFrame
            
            # Create iteration DataFrame to process each API response (up to 50 results)
            iter_df = pd.DataFrame(columns=['search_term']+cols)
            iter_df['search_term'] = [cat for i in range(len(response_df))] # Add category value for each row

            # Iterate through each requested column argument and format for storage in output DataFrame
            for col_name in cols:
                # Convert list of categories into single comma-separated string
                if col_name == 'categories':
                    # Exception handling: not all responses include all categories
                    try:
                        for k, cell in enumerate(response_df['categories']):
                            iter_cat_str = ''
                            for d in cell:
                                iter_cat_str += str(d['alias']+', ')
                            iter_df.loc[k, 'categories'] = iter_cat_str[:-2] # Save final string, without final ', ' 
                    except:
                        pass
                elif col_name in ('city','state','zip_code'): # Access location data through 'location' key value
                    try:
                        iter_df[col_name] = [response_df['location'][i][col_name] for i in range(response_df.shape[0])]
                    except:
                        pass
                elif col_name in ('latitude','longitude'): # Access latitude/longitude through 'coordinates' key value
                    try:
                        iter_df[col_name] = [response_df['coordinates'][i][col_name] for i in range(response_df.shape[0])]
                    except:
                        pass
                else:
                    try:
                        iter_df[col_name] = response_df[col_name] # Anything else access directly
                    except:
                        pass
            cat_df = cat_df.append(iter_df)
        output_df = output_df.append(cat_df)
    output_df.index = range(output_df.shape[0])
    
    return output_df


In [55]:
test_df = query_to_df('10128', limit_in=70, cat_in=['restaurants'])

In [56]:
test_df.head()

search_term                      categories  \
0  restaurants        catering, delis, grocery   
1  restaurants  wine_bars, southafrican, tapas   
2  restaurants                japanese, korean   
3  restaurants                           ramen   
4  restaurants                    tradamerican   

                            alias      city state zip_code price review_count  \
0      3rd-avenue-garden-new-york  New York    NY    10128    $$           15   
1          kaia-wine-bar-new-york  New York    NY    10128    $$          376   
2                  maroo-new-york  New York    NY    10128    $$          120   
3           naruto-ramen-new-york  New York    NY    10128    $$          853   
4  the-corner-restaurant-new-york  New York    NY    10128   $$$           13   

    latitude  longitude  
0  40.781930 -73.951940  
1  40.781900 -73.951970  
2  40.782476 -73.951333  
3  40.781170 -73.952500  
4  40.782630 -73.951210

In [4]:
test_df.shape

(70, 10)

In [5]:
test_df.groupby('search_term').price.value_counts()

search_term  price
restaurants  $$       47
             $        14
             $$$       5
Name: price, dtype: int64

In [7]:
test_df.groupby('search_term').zip_code.value_counts()

search_term  zip_code
restaurants  10128       69
             10028        1
Name: zip_code, dtype: int64

## API Pull from List of ZIP codes and categories

In [17]:
zip_list = ['10128','19025']
cats = ['restaurants, shopping, localservices']

## RESET RESULTS DATAFRAME `api_data`

In [13]:
api_data = pd.DataFrame(columns=['zip','city','state','cat','pr_1','rv_1','pr_2','rv_2','pr_3','rv_3','pr_4','rv_4','avg_lat','avg_long'])


In [14]:
api_data.head()

Empty DataFrame
Columns: [zip, city, state, cat, pr_1, rv_1, pr_2, rv_2, pr_3, rv_3, pr_4, rv_4, avg_lat, avg_long]
Index: []

In [15]:
def api_pull(zip_list, cats=[None], sort='best_match', limit=50):
    column_list = ['zip','city','state','cat',
                   'pr_1','rv_1','pr_2','rv_2',
                   'pr_3','rv_3','pr_4','rv_4',
                   'avg_lat','avg_long']
    
    api_data = pd.DataFrame(columns=column_list)
    
    for z in zip_list:
        df = query_to_df(z, cats, limit_in=limit, sort_in=sort)
        
        loop_df = pd.Series(index=column_list)

        loop_df['zip'] = z
        try:
            loop_df['city'] = df.city[0]
            loop_df['state'] = df.state[0]
        except: 
            pass

        loop_df['pr_1'] = df[df.price=='$'].shape[0]
        loop_df['rv_1'] = df[df.price=='$'].review_count.sum()
        loop_df['pr_2'] = df[df.price=='$$'].shape[0]
        loop_df['rv_2'] = df[df.price=='$$'].review_count.sum()
        loop_df['pr_3'] = df[df.price=='$$$'].shape[0]
        loop_df['rv_3'] = df[df.price=='$$$'].review_count.sum()
        loop_df['pr_4'] = df[df.price=='$$$$'].shape[0]
        loop_df['rv_4'] = df[df.price=='$$$$'].review_count.sum()

        loop_df['avg_lat'] = df.latitude.mean()
        loop_df['avg_long'] = df.longitude.mean()

        api_data = api_data.append(loop_df, ignore_index=True)
    
    api_data.zip = api_data.zip.astype(str)    
    return api_data

In [46]:
new_test = api_pull(zip_list, cats, limit=100)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/anaconda3/li

# Borough ZIP pull - 100 best match, no category

In [4]:
b_zips = pd.read_csv('../Data/nyc_borough_zip.csv')

In [5]:
bk_zips = b_zips[b_zips['brooklyn'].notnull()]['brooklyn']

In [6]:
bk_zips = bk_zips.astype(str).str.split('.',expand=True)[0]

In [7]:
qn_zips = b_zips[b_zips['queens'].notnull()]['queens']

In [8]:
qn_zips = qn_zips.astype(str).str.split('.',expand=True)[0]

In [9]:
bx_zips = b_zips[b_zips['bronx'].notnull()]['bronx']

In [10]:
bx_zips = bx_zips.astype(str).str.split('.',expand=True)[0]

In [11]:
si_zips = b_zips[b_zips['staten_island'].notnull()]['staten_island']

In [12]:
si_zips = si_zips.astype(str).str.split('.',expand=True)[0]

## Brooklyn

In [13]:
zips = bk_zips
cats = ['none']

In [16]:
yelp_bk = api_pull(zips, cats, limit=100)

In [17]:
yelp_bk.head()

zip           city state  cat pr_1  rv_1 pr_2   rv_2 pr_3  rv_3 pr_4  \
0  11201.0       Brooklyn    NY  NaN   15  6370   70  44823    8  2845    1   
1  11202.0       Brooklyn    NY  NaN    9   446    9    897    1   273    0   
2  11203.0  East Flatbush    NY  NaN   46  3072   49   5781    1   113    1   
3  11204.0       Brooklyn    NY  NaN   33  4587   62  17047    4   458    0   
4  11205.0       Brooklyn    NY  NaN   17  7784   70  43845   11  6466    1   

   rv_4    avg_lat   avg_long  
0   997  40.692694 -73.991436  
1     0  40.692161 -73.991179  
2    11  40.655470 -73.944190  
3     0  40.622603 -73.990887  
4  5131  40.697441 -73.967653

In [18]:
yelp_bk.shape

(47, 14)

## Queens

In [21]:
zips = qn_zips
cats = ['none']

In [22]:
yelp_qn = api_pull(zips, cats, limit=100)

In [23]:
yelp_qn.head()

zip           city state  cat pr_1   rv_1 pr_2   rv_2 pr_3   rv_3 pr_4  \
0  11004.0  New Hyde Park    NY  NaN   40   3091   56   8256    0      0    0   
1  11005.0        Astoria    NY  NaN   10    573    3    176    0      0    0   
2  11101.0       New York    NY  NaN   12  26372   61  82045    8  11472   11   
3  11102.0        Astoria    NY  NaN   21   3863   73  28359    2    325    0   
4  11103.0        Astoria    NY  NaN   24   5512   71  32584    4   1043    0   

    rv_4    avg_lat   avg_long  
0      0  40.744354 -73.715113  
1      0  40.748055 -73.734862  
2  24825  40.750680 -73.954138  
3      0  40.767802 -73.922745  
4      0  40.763566 -73.917122

## Bronx

In [24]:
zips = bx_zips
cats = ['none']

In [25]:
yelp_bx = api_pull(zips, cats, limit=100)

In [26]:
yelp_bx.head()

zip      city state  cat pr_1  rv_1 pr_2   rv_2 pr_3  rv_3 pr_4 rv_4  \
0  10451.0     Bronx    NY  NaN   32  2997   58  10723    4  3185    2  190   
1  10452.0     Bronx    NY  NaN   40   844   23   1066    5    75    1   86   
2  10453.0  New York    NY  NaN   29  3217   65  13757    2   562    0    0   
3  10454.0     Bronx    NY  NaN   37  2604   55   7539    3  1135    1  134   
4  10455.0  New York    NY  NaN   16  4205   72  39924    7  6018    1  134   

     avg_lat   avg_long  
0  40.817050 -73.932395  
1  40.834187 -73.920470  
2  40.858406 -73.923906  
3  40.804890 -73.930368  
4  40.801446 -73.940640

## Staten Island

In [27]:
zips = si_zips
cats = ['none']

In [28]:
yelp_si = api_pull(zips, cats, limit=100)

In [29]:
yelp_si.head()

zip           city state  cat pr_1  rv_1 pr_2   rv_2 pr_3  rv_3 pr_4  \
0  10301.0  Staten Island    NY  NaN   14  2226   69  23045    6   908    1   
1  10302.0  Staten Island    NY  NaN   29  3012   61   7383    3   466    0   
2  10303.0      Elizabeth    NJ  NaN   26  3405   70  13553    2   312    0   
3  10304.0  Staten Island    NY  NaN   17  2220   65  11378   11  1317    1   
4  10305.0  Staten Island    NY  NaN   13  6835   75  21257    9  1064    1   

  rv_4    avg_lat   avg_long  
0  176  40.631828 -74.069224  
1    0  40.633904 -74.121892  
2    0  40.635001 -74.142117  
3  176  40.614453 -74.093799  
4  176  40.612531 -74.064077

In [30]:
yelp_boroughs = yelp_bk

In [31]:
yelp_boroughs = yelp_boroughs.append(yelp_qn)

In [32]:
yelp_boroughs = yelp_boroughs.append(yelp_bx)

In [33]:
yelp_boroughs = yelp_boroughs.append(yelp_si)

In [34]:
yelp_boroughs.index = range(len(yelp_boroughs))

In [35]:
yelp_boroughs.zip = yelp_boroughs.zip.str.split('.', expand=True)[0]

In [36]:
yelp_boroughs.zip = yelp_boroughs.zip.map(lambda x: '0'+str(x) if int(x) <= 9999 else x)

In [37]:
yelp_boroughs

zip           city state  cat pr_1   rv_1 pr_2    rv_2 pr_3  rv_3 pr_4  \
0    11201       Brooklyn    NY  NaN   15   6370   70   44823    8  2845    1   
1    11202       Brooklyn    NY  NaN    9    446    9     897    1   273    0   
2    11203  East Flatbush    NY  NaN   46   3072   49    5781    1   113    1   
3    11204       Brooklyn    NY  NaN   33   4587   62   17047    4   458    0   
4    11205       Brooklyn    NY  NaN   17   7784   70   43845   11  6466    1   
5    11206       Brooklyn    NY  NaN   19   7690   73   44030    6  5780    2   
6    11207       Brooklyn    NY  NaN   31   6224   66   22526    1   245    0   
7    11208   Howard Beach    NY  NaN   35   4720   58   11148    3   870    1   
8    11209       Brooklyn    NY  NaN   16   2461   79   19560    4   678    0   
9    11210       Brooklyn    NY  NaN   30   2197   61   15455    5   562    0   
10   11211       Brooklyn    NY  NaN   20  21326   69  114276    8  9518    2   
11   11212       Brooklyn    NY  NaN   42   2587   49    5634    2   159    1   
12   11213       Brooklyn    NY  NaN   44   1796   40    1652    2   334    1   
13   11214       Brooklyn    NY  NaN   45   8114   51    9071    2   220    0   
14   11215       Brooklyn    NY  NaN   26   6133   67   28390    3  1392    0   
15   11216       Brooklyn    NY  NaN   23   6544   70   27031    4   998    0   
16   11217       Brooklyn    NY  NaN   12   3416   83   40313    4  1743    0   
17   11218       Brooklyn    NY  NaN   27   5165   70   20432    2    98    0   
18   11219       Brooklyn    NY  NaN   38   6880   57   10436    3   381    0   
19   11220       Brooklyn    NY  NaN   30   6264   68   15876    2   400    0   
20   11221       Brooklyn    NY  NaN   33   6123   66   24246    1   245    0   
21   11222       Brooklyn    NY  NaN   15   5759   74   46668    9  6979    1   
22   11223       Brooklyn    NY  NaN   37   8978   60   11858    2   259    1   
23   11224       Brooklyn    NY  NaN   31  10545   58   14232    4   606    0   
24   11225       Brooklyn    NY  NaN   39   3663   51    9282    2   288    1   
25   11226       Brooklyn    NY  NaN   26   3476   69   15083    2   216    0   
26   11228       Brooklyn    NY  NaN   22   3928   73   18909    3   537    0   
27   11229       Brooklyn    NY  NaN   30   4717   63   11439    4   651    1   
28   11230       Brooklyn    NY  NaN   28   2629   64   14661    6   584    0   
29   11231       Brooklyn    NY  NaN   17   5324   79   38121    2  1365    0   
..     ...            ...   ...  ...  ...    ...  ...     ...  ...   ...  ...   
134  10460          Bronx    NY  NaN   28   4192   65   13523    1   233    0   
135  10461          Bronx    NY  NaN   33   4035   60   11077    2   276    0   
136  10462          Bronx    NY  NaN   34   4328   60   11045    2   276    0   
137  10463          Bronx    NY  NaN   28   2609   64   12919    2   299    0   
138  10464          Bronx    NY  NaN   20   2967   66   15263   10  2390    0   
139  10465          Bronx    NY  NaN   31   3529   61   11361    7  1987    0   
140  10466   Mount Vernon    NY  NaN   32   2331   63    8906    2   317    0   
141  10467          Bronx    NY  NaN   31   3817   61   13570    2   439    0   
142  10468          Bronx    NY  NaN   27   2871   70   14008    1   233    0   
143  10469          Bronx    NY  NaN   44   2584   47    3691    0     0    1   
144  10470          Bronx    NY  NaN   51   1245   36    2223    2    75    0   
145  10471          Bronx    NY  NaN   20   2443   69   13493    3  1030    0   
146  10472          Bronx    NY  NaN   32   4242   64   11895    1   233    0   
147  10473          Bronx    NY  NaN   40   5319   57   12548    1   233    0   
148  10474        Astoria    NY  NaN   26   5910   67   21160    2  1110    1   
149  10475          Bronx    NY  NaN   35   3906   56    9598    5  1797    0   
150  10301  Staten Island    NY  NaN   14   2226   69   23045    6   908    1   
151  10302  Staten Island    NY  NaN  

In [38]:
nyc_zips = pd.read_csv('../nyc_zip.csv', header=None, names=['zip'], dtype={'zip':str})

In [39]:
zips = nyc_zips.zip
cats = ['none']

In [40]:
yelp_manh_1 = api_pull(zips[:80], cats, limit=100)

In [41]:
yelp_manh_2 = api_pull(zips[80:], cats, limit=100)

In [42]:
yelp_manh = yelp_manh_1.append(yelp_manh_2, ignore_index=True)

In [43]:
yelp_manh.reindex(axis=0)

zip        city state  cat pr_1   rv_1 pr_2    rv_2 pr_3   rv_3 pr_4  \
0    10001.0    New York    NY  NaN   17  20139   64   73672   12  13352    4   
1    10002.0    New York    NY  NaN   29  26811   64   87914    6   5705    0   
2    10003.0    New York    NY  NaN   21  28355   64  115635    9  13478    5   
3    10004.0    New York    NY  NaN   21  38197   52  113225   13  16930    5   
4    10005.0    New York    NY  NaN   22   4917   61   21109    9   2624    3   
5    10006.0    New York    NY  NaN   25   5000   57   15581    6   1506    4   
6    10007.0    New York    NY  NaN   21   4489   56   16660   12   5307    3   
7    10008.0    Amarillo    TX  NaN   25   1048   16    1752    1      5    0   
8    10009.0    New York    NY  NaN   22  25276   66  103451   10  11673    2   
9    10010.0    New York    NY  NaN   13  15800   55   85243   26  27691    4   
10   10011.0    New York    NY  NaN   19  40897   51   89139   20  31188    8   
11   10012.0    New York    NY  NaN   17  21994   56  105487   22  26700    4   
12   10013.0    New York    NY  NaN   19  27788   48   65226   27  28513    5   
13   10014.0    New York    NY  NaN   15  18600   49   52722   30  33949    5   
14   10015.0      Tucson    AZ  NaN   37   7257   59   18846    1    337    0   
15   10016.0    New York    NY  NaN    7   6226   62   63120   18  19892    7   
16   10017.0    New York    NY  NaN   15  20883   47   39323   20  17671   11   
17   10018.0    New York    NY  NaN   16  22434   67   77687    8   6440    6   
18   10019.0    New York    NY  NaN   13  25010   54   79356   17  18360    8   
19   10020.0    New York    NY  NaN   26   3321   45   16012   16   6846    7   
20   10021.0    New York    NY  NaN   14   4155   66   36569   11  10675    8   
21   10022.0    New York    NY  NaN   13  17697   48   34872   24  21003    9   
22   10023.0    New York    NY  NaN   12   5020   50   38010   23  13911    9   
23   10024.0    New York    NY  NaN   17   6519   72   56887    6   2340    0   
24   10025.0    New York    NY  NaN   19   6757   71   49187    4   4018    0   
25   10026.0    New York    NY  NaN   14   3405   78   29849    4   3236    0   
26   10027.0      Harlem    NY  NaN   11   2257   79   28835    5   3321    1   
27   10028.0    New York    NY  NaN   10   3492   73   40192    7   6153    5   
28   10029.0    New York    NY  NaN   36   3786   55   11893    4   1465    1   
29   10030.0    New York    NY  NaN   31   2753   63   11380    3    344    1   
..       ...         ...   ...  ...  ...    ...  ...     ...  ...    ...  ...   
135  10211.0      Downey    CA  NaN   46  27128   50   39587    2   1955    0   
136  10212.0     Seattle    WA  NaN   39  21535   53   27889    4   2879    2   
137  10213.0    New York    NY  NaN    8    888   41   14242   34   9753    9   
138  10242.0    New York    NY  NaN   17   3918   55   18415   15   6627    6   
139  10249.0    New York    NY  NaN   17   3918   55   18415   15   6627    6   
140  10256.0    New York    NY  NaN   17   3918   55   18415   15   6627    6   
141  10257.0     Chimayo    NM  NaN   28    389   19    1139    3     76    1   
142  10258.0    Carlsbad    NM  NaN   38    983   18    1365    1      1    1   
143  10259.0    New York    NY  NaN   17   3918   55   18415   15   6627    6   
144  10260.0      Tigard    OR  NaN   48  10266   51   17544    0      0    0   
145  10261.0    New York    NY  NaN   17   3918   55   18415   15   6627    6   
146  10265.0    New York    NY  NaN   25   5209   58   19820    9   2984    4   
147  10268.0    New York    NY  NaN   18   4252   60   19920   14   6033    3   
148  10269.0     Houston    TX  NaN   33  10518   58   26808    5   2077    2   
149  10270.0  Louisville    KY  NaN   23   4151   55   19509   13   3162    2   
150  10271.0    New York    NY  NaN    5    311    2     224    1    523    0   
151  10272.0    New York    NY  NaN    5    486    3     559    1    433    1   
152  10273.0    New York    NY  NaN   25

In [44]:
yelp_manh.zip = yelp_manh.zip.str.split('.', expand=True)[0]

In [45]:
yelp_manh.zip = yelp_manh.zip.map(lambda x: '0'+str(x) if int(x) <= 9999 else x)

In [61]:
nyc_best = yelp_manh.append(yelp_boroughs)

In [62]:
nyc_best.head()

zip      city state  cat pr_1   rv_1 pr_2    rv_2 pr_3   rv_3 pr_4  rv_4  \
0  10001  New York    NY  NaN   17  20139   64   73672   12  13352    4  6783   
1  10002  New York    NY  NaN   29  26811   64   87914    6   5705    0     0   
2  10003  New York    NY  NaN   21  28355   64  115635    9  13478    5  7834   
3  10004  New York    NY  NaN   21  38197   52  113225   13  16930    5  5147   
4  10005  New York    NY  NaN   22   4917   61   21109    9   2624    3  1330   

     avg_lat   avg_long  
0  40.751380 -73.991945  
1  40.719674 -73.989291  
2  40.730335 -73.989247  
3  40.715223 -73.999469  
4  40.706818 -74.008856

In [63]:
nyc_best.shape

(329, 14)

In [57]:
nyc_best.drop(['cat'], axis=1, inplace=True)

In [58]:
nyc_best[nyc_best.state!='NY']

zip                city state pr_1   rv_1 pr_2   rv_2 pr_3   rv_3 pr_4  \
7    10008            Amarillo    TX   25   1048   16   1752    1      5    0   
14   10015              Tucson    AZ   37   7257   59  18846    1    337    0   
45   10047             Houston    TX   52  15861   45  18992    2   1038    0   
46   10048            Portales    NM   11    153   10    239    0      0    0   
48   10060            Manassas    VA   40   3910   52   8331    3    730    0   
51   10072        Garden Grove    CA   41  41873   45  54055    6  10282    2   
54   10080       Overland Park    KS   29   4570   63  14546    5   1148    1   
58   10090           San Diego    CA   54  18064   42  17596    0      0    0   
59   10094  Indian Rocks Beach    FL   37   4843   55  13662    3   1030    0   
63   10099        Bartlesville    OK   35    430   22    668    5     57    0   
64   10101          Louisville    KY   25   2065   71  12247    3    323    0   
77   10114               Tulsa    OK   29   1739   60   5870    6    658    1   
83   10120           Richfield    MN   32   7066   62  18777    2    384    0   
87   10124          Las Cruces    NM   57   3397   38   4745    0      0    0   
89   10126            Portales    NM   11    153   10    239    0      0    0   
92   10130             Houston    TX   46    542   19    394    2      6    0   
93   10131       Palos Heights    IL   41   2325   51   5669    2    176    0   
94   10132              Stuart    FL   23    971   65   6224    7    808    0   
95   10133            Presidio    TX    6    154    0      0    0      0    0   
99   10151              Kenner    LA   38   3209   60   6862    1     79    0   
104  10156          Cincinnati    OH   41   3794   51   8259    2    645    1   
105  10157             Houston    TX   25   4629   63  18546   10   4178    1   
107  10159               Tulsa    OK   54   2862   36   3280    6    740    0   
108  10160         Bonne Terre    MO   37    376   34    653    0      0    0   
109  10161            Montreal    QC   18   5126   54  19733   19   5968    5   
129  10185           Cupertino    CA   34  26117   61  61499    3   2628    1   
130  10196               Hobbs    NM   12     98   29    557    1     14    0   
133  10200               Hobbs    NM   24    161   34    611    1     14    0   
134  10203            Eastover    SC    9     78    3     26    0      0    0   
135  10211              Downey    CA   46  27128   50  39587    2   1955    0   
136  10212             Seattle    WA   39  21535   53  27889    4   2879    2   
141  10257             Chimayo    NM   28    389   19   1139    3     76    1   
142  10258            Carlsbad    NM   38    983   18   1365    1      1    1   
144  10260              Tigard    OR   48  10266   51  17544    0      0    0   
148  10269             Houston    TX   33  10518   58  26808    5   2077    2   
149  10270          Louisville    KY   23   4151   55  19509   13   3162    2   
154  10275             Arcadia    LA   38    476   36    771    0      0    0   
44   11251             Orlando    FL   17   7236   62  38497   13   7914    1   
45   11252               Salem    WI   34   1948   60   6816    4    622    0   
88   11405             El Paso    TX   48   2700   50   5682    1    162    0   
152  10303           Elizabeth    NJ   26   3405   70  13553    2    312    0   
163  10314           Elizabeth    NJ   18   2151   77  13639    3    948    0   

     rv_4    avg_lat    avg_long  
7       0  35.205871 -101.771002  
14      0  32.202802 -110.958123  
45      0  29.714243  -95.546373  
46      0  34.183415 -103.339380  
48      0  38.756750  -77.466215  
51   2398  33.784637 -117.946202  
54    127  38.943825  -94.676917  
58      0  32.910766 -117.121253  
59      0  27.887151  -82.816396  
63      0  36.741756  -95.949024  
64      0  38.253537  -85.589795  
77     83  36.042610  -95.908609  
83      0  44.867106  -93.282037  
87      0  32.299925 -106.772817 

In [64]:
nyc_best = nyc_best[nyc_best.state=='NY']

In [70]:
nyc_best[(nyc_best.avg_long>-73.5)|(nyc_best.avg_lat>40.95)]

zip           city state  cat pr_1  rv_1 pr_2  rv_2 pr_3 rv_3 pr_4  \
50   10069       New York    NY  NaN    4   250    6   770    0    0    0   
110  10162       New York    NY  NaN    6   489   12  1258    1  274    2   
113  10165       New York    NY  NaN    3   136    7   444    2  554    2   
114  10166       New York    NY  NaN   16  1098   36  3963    5  816    1   
125  10177       New York    NY  NaN    8   548   14  1376    0    0    0   
160  10281       New York    NY  NaN    4   296    5   375    3  250    0   
115  11439  Fresh Meadows    NY  NaN   15   838    9  1028    0    0    0   

     rv_4    avg_lat   avg_long  
50      0  41.460600 -70.349319  
110  1156  41.325086 -71.051391  
113  1212  41.438608 -70.803575  
114   110  40.763555 -72.919105  
125     0  40.778229 -71.351032  
160     0  41.316645 -69.616675  
115     0  41.197305 -71.172356

In [76]:
nyc_best = nyc_best[(nyc_best.avg_long<-73.5)&(nyc_best.avg_lat<40.95)&(nyc_best.avg_long>-74.3)]

In [77]:
nyc_best.index = range(nyc_best.shape[0])

In [78]:
nyc_best.describe()

cat     avg_lat    avg_long
count  0.0  278.000000  278.000000
mean   NaN   40.730332  -73.938603
std    NaN    0.066512    0.087198
min    NaN   40.516515  -74.244572
25%    NaN   40.700638  -73.991050
50%    NaN   40.740227  -73.962156
75%    NaN   40.761186  -73.886046
max    NaN   40.897152  -73.715113

In [79]:
nyc_best.to_csv('../Data/nyc_best.csv', index=False)

ZIP Code Database: https://www.unitedstateszipcodes.org/zip-code-database/

In [53]:
zip_db = pd.read_csv('../Data/zip_code_database.csv', dtype={'zip':str})

In [24]:
zip_db.head()

zip      type  decommissioned primary_city acceptable_cities  \
0  00501    UNIQUE               0   Holtsville               NaN   
1  00544    UNIQUE               0   Holtsville               NaN   
2  00601  STANDARD               0     Adjuntas               NaN   
3  00602  STANDARD               0       Aguada               NaN   
4  00603  STANDARD               0    Aguadilla             Ramey   

                                 unacceptable_cities state  \
0                               I R S Service Center    NY   
1                                 Irs Service Center    NY   
2  Colinas Del Gigante, Jard De Adjuntas, Urb San...    PR   
3  Alts De Aguada, Bo Guaniquilla, Comunidad Las ...    PR   
4  Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...    PR   

                county             timezone area_codes world_region country  \
0       Suffolk County     America/New_York        631          NaN      US   
1       Suffolk County     America/New_York        631          NaN      US   
2   Adjuntas Municipio  America/Puerto_Rico    787,939          NaN      US   
3     Aguada Municipio  America/Puerto_Rico    787,939          NaN      US   
4  Aguadilla Municipio  America/Puerto_Rico        787          NaN      US   

   latitude  longitude  irs_estimated_population_2015  
0     40.81     -73.04                            562  
1     40.81     -73.04                              0  
2     18.16     -66.72                              0  
3     18.38     -67.18                              0  
4     18.43     -67.15                              0

In [54]:
lat_map = {zip_db.zip[i]: zip_db.latitude[i] for i in range(zip_db.shape[0])}
long_map = {zip_db.zip[i]: zip_db.longitude[i] for i in range(zip_db.shape[0])}

In [63]:
yelp_b = yelp_boroughs[yelp_boroughs.pr_1 + yelp_boroughs.pr_2 + yelp_boroughs.pr_3 + yelp_boroughs.pr_4 > 0]

In [64]:
yelp_b['state'] = 'NY'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
city_ref_dict = {zip_db.zip[i]: zip_db.primary_city[i] for i in range(zip_db.shape[0])}

In [66]:
city_ref_dict['11204']

'Brooklyn'

In [71]:
yelp_b['db_city'] = yelp_b.zip.apply(lambda x: city_ref_dict[x])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
yelp_b.city = yelp_b.db_city

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [74]:
yelp_b.drop('db_city', axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [75]:
yelp_b.head()

zip      city state   cat pr_1  rv_1 pr_2   rv_2 pr_3  rv_3 pr_4 rv_4  \
0  11201  Brooklyn    NY  none   13  4563   49  25627    7  2308    1  998   
2  11203  Brooklyn    NY  none   10   649    9   2225    0     0    0    0   
3  11204  Brooklyn    NY  none   13  1527   17   2684    1    77    0    0   
4  11205  Brooklyn    NY  none    8  2994   18   5987    3   365    0    0   
5  11206  Brooklyn    NY  none    7  1938   20   8772    0     0    1  162   

     avg_lat   avg_long  
0  40.693368 -73.991550  
2  40.650264 -73.933248  
3  40.615993 -73.988200  
4  40.692484 -73.966175  
5  40.705367 -73.940534

In [76]:
yelp_b.to_csv('../Data/yelp_b.csv', index=False)

In [77]:
yelp_manh = pd.read_csv('../Data/yelp.csv')

In [78]:
yelp_manh.head()

zip      city state   cat  pr_1   rv_1  pr_2   rv_2  pr_3  rv_3  pr_4  \
0  10001  New York    NY  none     8   9740    28  29095     4  5507     0   
1  10002  New York    NY  none    18  15201    39  43425     4  4740     0   
2  10003  New York    NY  none    16  19139    54  91952     6  8836     3   
3  10004  New York    NY  none    12   1980    26  10361     6  2128     2   
4  10005  New York    NY  none     7   1425    11   2268     1    89     1   

   rv_4    avg_lat   avg_long  
0     0  40.747709 -73.990216  
1     0  40.719057 -73.989387  
2  5125  40.730866 -73.988554  
3  1153  40.704432 -74.011839  
4   177  40.706222 -74.008576

In [79]:
yelp_nyc = yelp_manh

In [80]:
yelp_nyc = yelp_nyc.append(yelp_b)

In [81]:
yelp_nyc.shape

(210, 14)

In [82]:
yelp_nyc.to_csv('../Data/yelp_nyc_total.csv', index=False)